In [1]:
import PySpin
import matplotlib.pyplot as plt
import cv2

In [2]:
system = PySpin.System.GetInstance()
camera_list = system.GetCameras()
num_cam = camera_list.GetSize()
num_cam
cam = camera_list.GetByIndex(0)
cam.Init()
cam.TLStream.StreamBufferHandlingMode.SetValue(
PySpin.StreamBufferHandlingMode_NewestOnly
)

if not cam.IsStreaming():
    cam.BeginAcquisition()

### Get Image

In [21]:
grabTimeout: int = PySpin.EVENT_TIMEOUT_INFINITE
streamID: int = 0

pyspin_image = cam.GetNextImage(grabTimeout, streamID)
image_array = pyspin_image.GetNDArray()

### Convert to RGB

In [23]:
img = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)

### Resize

In [26]:
img = cv2.resize(img, (16,16))

### Visualize

In [27]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image', img)
cv2.waitKey(0)

-1